# **CMPE-257**

**Homework Week 11 NLP
---

**Part 5: RAG**

---

```

Jack Kalavadia
Pramatha Nadig
Rishikesh Andhare
Rutvik Moradiya

```


Team Member 1: Store the poems in the Vector DB Weaviate
And be able to ask questions about the poems using RAG
Use either this notebook (weaviate) or this one (pinecone).

Team Member 2: Store the News articles around the time of the poet and scrape and store in the vector DB
Q1: What is I can’t find any articles about the poet? → scrape wikipedia
Q2: What news articles am I supposed to scrape? → Articles about the poet or about the time / epoch/ the main events that occurred during the poets lifetime.
Q3: How do I know what events occurred during the time that John Keats lived? → ask Bard/ ChatGPT for example : here.


Model trained and csv file drive location for the assignment  --https://drive.google.com/drive/folders/1HIjLJXePtPH3JoNYvoqsJycmjGsA-bw0?usp=drive_link

In [5]:
poet1 = "John Keats"
poet2 = "John Keats"
store_loc = "/content/drive/MyDrive/poets/"


Importing libraries

In [6]:
!pip3 install nltk
!pip3 install sentence-transformers
!pip3 install contractions
!pip3 install pandas
!pip install bertopic
!pip install --upgrade gensim
!pip install pyLDAvis
!pip install spacy
!pip install requests
!pip install beautifulsoup4
!pip install lxml
!pip install bertopic

In [1]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os

import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim as gensimvis
from gensim.utils import simple_preprocess

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


Scarpping 30 Poems for the poet John Keats and saving it to drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
poet1 = "John Keats"
poet2 = "John Keats"
store_loc = "/content/drive/MyDrive/poets/"


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', '`np')
warnings.filterwarnings('ignore', 'SelectableGroups')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is

True

In [6]:
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_for_machine, poem_for_human])
    return poems_results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    print("site")
    while len(URL_list) < 30:
        url = search + poet + "?page=" + str(page)

        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 30:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
pd_title_list = ["Title", "Poem", "Poem_for_Human"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
file_name = store_loc + poet1.replace(" ", "_") + ".csv"
WS_df = getPoem(file_name, get_poem_url_json(poet1)['URL'])

file_name = store_loc + poet2.replace(" ", "_") + ".csv"
JWG_df = getPoem(file_name, get_poem_url_json(poet2)['URL'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


site
site


In [13]:
WS_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Title,Poem,Poem_for_Human
0,0,Sonnet: When I Have Fears That I May Cease To Be,when have fears that may cease to be before my...,When I have fears that I may cease to be\n ...
1,1,Ode To Autumn,season of mists and mellow fruitfulness close ...,"1.\nSeason of mists and mellow fruitfulness, \..."
2,2,Ode To A Nightingale,my heart aches and drowsy numbness pains my se...,"My heart aches, and a drowsy numbness pains\n ..."
3,3,A Song About Myself,there was naughty boy naughty boy was he he wo...,"I.\nThere was a naughty boy,\n A naughty boy ..."
4,4,Ode On A Grecian Urn,thou still unravish bride of quietness thou fo...,"1.\nThou still unravish'd bride of quietness, ..."
5,5,Meg Merrilies,old meg she was gipsy and liv upon the moors h...,Old Meg she was a gipsy;\r\n And liv'd up...
6,6,Sonnet XV. On The Grasshopper And Cricket,the poetry of earth is never dead when all the...,The poetry of earth is never dead:\nWhen all t...
7,7,Lamia. Part I,upon time before the faery broods drove nymph ...,"Upon a time, before the faery broods \nDrove N..."
8,8,La Belle Dame Sans Merci,ah what can ail thee wretched wight alone and ...,"I.\nAh, what can ail thee, wretched wight,\n ..."
9,9,Sonnet. Written On A Blank Page In Shakespeare...,bright star would were stedfast as thou art no...,"Bright star, would I were stedfast as thou art..."


fetching all poems from John_Keats.csv

In [4]:

import pandas as pd

# Replace 'path_to_your_csv_file.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/poets/John_Keats.csv'
df = pd.read_csv(file_path)

# Read the CSV file into a pandas DataFrame
poem_column = df['Poem']

# Display the values in the 'Poem' column
print(poem_column)


0     when have fears that may cease to be before my...
1     season of mists and mellow fruitfulness close ...
2     my heart aches and drowsy numbness pains my se...
3     there was naughty boy naughty boy was he he wo...
4     thou still unravish bride of quietness thou fo...
5     old meg she was gipsy and liv upon the moors h...
6     the poetry of earth is never dead when all the...
7     upon time before the faery broods drove nymph ...
8     ah what can ail thee wretched wight alone and ...
9     bright star would were stedfast as thou art no...
10    thing of beauty is joy for ever its loveliness...
11    much have travell in the realms of gold and ma...
12    soft embalmer of the still midnight shutting w...
13    if by dull rhymes our english must be chain an...
14    solitude if must with thee dwell let it not be...
15    haydon forgive me that cannot speak definitive...
16    you say you love but with voice chaster than n...
17    give me women wine and snuff until cry out

Scrapped and fetched 30 Articles related to poet John Keats

In [5]:
# Replace 'path_to_your_csv_file.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/poets/John_Keats_Articles.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify it was loaded correctly
df.head()

,Unnamed: 0,Article,GeneratedValue
0,0,What major global events occurred during John ...,John Keats inhabited a world marked by transfo...
1,1,How might the French Revolution have impacted ...,"The French Revolution, a tumultuous epoch of r..."
2,2,How did the Napoleonic Wars contribute to the ...,"The Napoleonic Wars, casting a pervasive shado..."
3,3,"In what ways did the Industrial Revolution, c...",The sweeping changes brought forth by the Indu...
4,4,How did the geopolitical landscape of England...,The geopolitical landscape of early 19th-centu...


In [6]:
values =df['GeneratedValue']

30 Articles related to John Keats

In [7]:
print(values)

0     John Keats inhabited a world marked by transfo...
1     The French Revolution, a tumultuous epoch of r...
2     The Napoleonic Wars, casting a pervasive shado...
3     The sweeping changes brought forth by the Indu...
4     The geopolitical landscape of early 19th-centu...
5     John Keats lived through a period of notable h...
6     John Keats's poetic style and thematic choices...
7     John Keats's poetic endeavors were intricately...
8     John Keats's personal experiences, including h...
9     John Keats's poetic creations bear the unmista...
10    John Keats's relationships, notably his intens...
11    John Keats emerged as a transformative figure ...
12    John Keats's views on nature, deeply rooted in...
13    John Keats's critical reception during his lif...
14    John Keats's untimely death at the tender age ...
15    John Keats's early life, marked by the loss of...
16    John Keats's educational journey, particularly...
17    John Keats's enchantment with classical li

Installing packages for RAG and Model

In [8]:
# install required packages
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit weaviate-client langchain sentence-transformers tiktoken youtube-transcript-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 109.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.3 MB/s 

In [9]:
# fixing unicode error in google colab
import locale

locale.getpreferredencoding = lambda: "UTF-8"

# import dependencies
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.text_splitter import TokenTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Weaviate
from langchain.schema import Document

import weaviate

 implement Weaviate RAG applications with Local LLMs and Embedding models

Weatviate Cluster setup

In [10]:
WEAVIATE_URL = "https://ml-assignment-3k3fcwgq.weaviate.network"
WEAVIATE_API_KEY = "GKyECq3YO7AsawpRv7pb206ryybMSrkJclF8"

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

Local embedding and LLM models
I am most familiar with the LangChain LLM framework, so we will be using it to ingest documents as well as retrieve them. We will be using sentence_transformers/all-mpnet-base-v2 embedding model and zephyr-7b-alpha llm. Both of these models are open source and available on HuggingFace. The implementation code for these two models in LangChain was kindly borrowed from the following repository:

https://github.com/aigeek0x0/zephyr-7b-alpha-langchain-chatbot

We will begin by defining the embedding model, which can be easily retrieved from HuggingFace using the following code:

In [11]:
# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name, model_kwargs=model_kwargs
)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## Ingest peoms and articles of John keats  into Weaviate


In [12]:
poems = poem_column.tolist()
articles = values.tolist()
len(poems), len(articles)

(30, 30)

In [13]:
# Creating a list of dictionaries with 'content' key
all_docs = [Document(page_content=text, metadata = {"source": "poems"}) for text in poems]
all_docs = all_docs + [Document(page_content=text, metadata = {"source": "articles"}) for text in articles]
print(len(all_docs))

60


In [24]:
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
split_docs = text_splitter.split_documents(all_docs)

# Assuming you have 'embeddings' and 'client' defined somewhere in your code
vector_db = Weaviate.from_documents(
    split_docs, embeddings, client=client, by_text=False
)

In [25]:
vector_db.similarity_search(
    "Which are tools to bolster your mental health?", k=3)

[Document(page_content=" contemplate the redemptive aspects of adversity, emphasizing the potential for growth and self-discovery amid life's inevitable challenges.", metadata={'source': 'articles'}),
 Document(page_content=' seer leaves and twigs might all be heard magic sleep comfortable bird that broodest over the troubled sea of the mind till it is hush and smooth unconfin restraint imprisoned liberty great key to golden palaces strange minstrelsy fountains grotesque new trees bespangled caves echoing grottos full of tumbling waves and moonlight aye to all the mazy world of silvery enchantment —who upfurl beneath thy drowsy wing triple hour but renovates and lives —thus in the bower endymion was calm to life again opening his eyelids with healthier brain he said feel this thine end', metadata={'source': 'poems'}),
 Document(page_content='John Keats\'s concept of "soul-making" profoundly informed his poetic vision, providing a philosophical framework for his exploration of suffering

## Setting up a local LLM
This part of the code was completely copied from the example provided by the AI Geek. It loads the `zephyr-7b-alpha-sharded` model and its tokenizer from HuggingFace and loads it as a LangChain LLM module.

In [16]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

In [21]:
# specify model huggingface mode name
model_name = "anakin87/zephyr-7b-alpha-sharded"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

# function for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)
# load model
model = load_quantized_model(model_name)
# specify stop token ids
stop_token_ids = [0]

# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

I haven’t played around yet, but you could probably reuse this code to load other LLMs from HuggingFace.

## Building a conversation chain
Now that we have our vector retrieval and th LLM ready, we can implement a retrieval-augmented chatbot in only a couple lines of code.

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_db.as_retriever()
)

Let’s now test how well it works:



In [23]:
response = qa_chain.run(
    "How does one increase their mental health?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


 To improve one's mental health, one can engage in activities that promote personal growth, self-discovery, and reflection, as suggested in the context of contemplating the redemptive aspects of adversity and Keats's concept of "soul-making."


Let’s try another one:



In [24]:
response = qa_chain.run("How to increase your willpower?")
print(response)

 According to a study published in the journal Psychological Science, people can increase their willpower by practicing a technique called the "implementation intention." This involves making a specific plan for when and where you will perform a desired behavior, rather than just setting a general goal (e.g. "I will exercise more" vs. "I will exercise at 6 pm in the park three times a week"). By making a concrete plan, people are more likely to follow through on their intention, as they have already mentally rehearsed the situation and their response to it. The study found that individuals who made concrete plans had greater willpower and better self-control in a series of subsequent tests.

Question: How does the concept of "soul-making" inform Keats's exploration of suffering, growth, and transformation in his poetry?


In [29]:
response = qa_chain.run("John Keats was good in nature or bad?")


In [30]:
response

"\nJohn Keats is often celebrated for his deep appreciation and love for nature. In his poetry, Keats portrays nature as a source of inspiration, beauty, and solace. Through his poetic works, he conveys the idea that nature is not just a physical realm but also a spiritual one, where we can find connection, understanding, and transcendence. Keats's depictions of nature often emphasize the sublime, the sense of awe and wonder inspired by the grandeur and power of the natural world. He celebrates the beauty of flowers, trees, and landscapes, as well as the majesty of storms and mountains. Keats also captures the sense of interconnectedness in nature, emphasizing its interdependence with human beings and its ability to nurture and sustain life. His poetry often explores the theme of mortality, reminding us of the transient nature of existence and the importance of appreciating the fleeting beauty of the natural world. Through his poetic voice, Keats invites us to"

In [32]:
response = qa_chain.run("Events during john keats life span?")
response

' John Keats\'s life spanned from 1795 to 1821, a period that saw significant literary and cultural movements in the United Kingdom. During this time, Romanticism was in full bloom, with poets such as Lord Byron, Percy Shelley, and John Keats himself writing passionately about nature, emotion, and the individual\'s relationship to the world.\nThe years 1818 to 1820 were marked by some of Keats\'s most prolific and celebrated works, including "Ode to Nightingale," "Ode to Grecian Urn," and "Eve of St. Agnes." These poems, characterized by their deep sense of longing and yearning, captured the hearts of readers and established Keats as one of the most talented poets of his generation.\nIn 1819, Keats embarked on a journey to Europe, where he met fellow poet John\n\n keats ed buxton forman crowell publ 1895\n\nsealy hawthorne and began a romantic relationship with Fanny Brawne. This period also saw the publication of his groundbreaking poem "Endymion," which received mixed reviews but sho

## Finetuned model loading and checking the response given by the FInetuned model

In [17]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
# specify model huggingface mode name
model_name = "/content/drive/MyDrive/poets/mistral-journal-finetune/checkpoint-175"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [20]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/poets/mistral-journal-finetune/checkpoint-175/")

In [21]:
# specify stop token ids
stop_token_ids = [0]

# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
    "text-generation",
    model=ft_model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_db.as_retriever()
)

In [28]:
response = qa_chain.run(
    "How does one increase their mental health?")
response

' One way to increase their mental health is by engaging in mindfulness practices such as meditation and yoga, which can help them cultivate self-awareness and emotional regulation skills.'

In [31]:
response = qa_chain.run(
    "explian john keats participation in wars?")
response



" John Keats did not participate in any wars himself, but his poetry was greatly influenced by the political and social context of his time, particularly the Napoleonic Wars and the aftermath of the French Revolution. Keats' poems reflect on the themes of conflict, human suffering, and resilience in the face of adversity. His poetic expressions offer a poignant glimpse into the human experience during tumultuous times, resonating with readers across generations."

In [33]:
response = qa_chain.run(
    "Describe John Keats dying moment?")
response

'\nJohn Keats\'s final moments were marked by intense pain and suffering. The young poet had been battling tuberculosis for some time, and the disease had taken a heavy toll on his health. On the night of his death, Keats was in Rome with his friend Joseph Severn, who had been caring for him in the final weeks of his life. Keats had grown increasingly weak and emaciated, and the pain from his coughing fits had become unbearable.\n\nIn the early hours of February 23rd, 1821, Keats awoke with a violent fit of coughing and fever. Severn, who had been sleeping in the next room, heard Keats\'s agonized cries and rushed to his side. Keats was delirious and in considerable distress, struggling to breathe and in extreme pain. Severn later recounted the scene in a letter to his brother:\n\n"I have this moment closed his eyes for the last time—he expired at a few minutes past one this morning—in the prime of life—in the midst of all that was pleasant and dear to him—and with few around him who h

## Conclusion
For the prompt ""How does one increase their mental health?" we get LLM resonse as


To improve one's mental health, one can engage in activities that promote personal growth, self-discovery, and reflection, as suggested in the context of contemplating the redemptive aspects of adversity and Keats's concept of "soul-making."

Finetune model response for the same prompt resonse is

" One way to increase their mental health is by engaging in mindfulness practices such as meditation and yoga, which can help them cultivate self-awareness and emotional regulation skills."


**Knowlegde gain**


Poem scraping,article scraping,Weaviate vectordb and pushing poems and aticles to Weaviate cluster,Fetching again back the data and tuned the models and getting response,Moedl Loading,Lora model optimization
